In [1]:
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [2]:
load_dotenv()
gpt4o = ChatOpenAI(model_name="gpt-4o-2024-08-06") 

In [3]:
supported_keywords: list[str] = ["프론트엔드", "백엔드", "클라우드", "보안", "MLOps"]
all_keywords = ", ".join(supported_keywords)

In [4]:
keyword_chain_template = PromptTemplate.from_template(
    f"""
    사용자 입력에서 {{all_keywords}}에 해당하는 키워드를 추출하세요.
    반환 형태를 list[str] 형태를 JSON 형식으로 출력하세요.

    입력:
    나는 프론트엔드와 백엔드에 관심이 많아

    키워드:
    ["프론트엔드", "백엔드"]

    입력:
    나는 정보 보안에 관심이 많은데, 클라우드 보안을 배워보고싶어!

    키워드:
    ["보안", "클라우드"]

    입력:
    {{user_input}}

    키워드:
    """.strip()
)

In [5]:
def get_keywords_from_input(user_input: str) -> str:
    all_keywords_str = ", ".join(all_keywords)
    result = keyword_chain_template | gpt4o | JsonOutputParser()
    extracted_keywords = result.invoke({"user_input": user_input, "all_keywords": all_keywords_str})
    return extracted_keywords

In [6]:
default_user_input = "나는 MLOps에 관심이 있어!"
user_input = input().strip() or default_user_input
keywords = get_keywords_from_input(user_input)
keywords

['MLOps']

In [7]:
keywords_str = ", ".join(keywords)
if len(keywords) > 1:
    print(f"다방면으로 관심이 많으시군요! 아쉽게도 여러 분야 책을 한번에 추천드릴 수가 없네요. {keywords_str} 중 사용자의 성향에 더 적합한 직무의 책을 추천하기 위해 몇가지 질문을 드려볼게요")
elif len(keywords) == 1:
    print(f"최근 각광받고 있는 {keywords_str}에 관심이 있다니 멋지네요! 관련 지식수준을 알아볼까요?")
else:
    print("관심 분야를 추출할 수 없습니다. 프론트엔드/백엔드/클라우드/보안 4가지에 한정하여 관심 분야를 알려주세요. 책 추천 받기를 원치 않으시다면 “그만”이라고 써주세요.")

최근 각광받고 있는 MLOps에 관심이 있다니 멋지네요! 관련 지식수준을 알아볼까요?


In [8]:
from dataclasses import dataclass

OptionValue = str

@dataclass
class JobOptions:
    frontend: OptionValue
    backend: OptionValue
    cloud: OptionValue
    security: OptionValue

@dataclass
class QuestionOptionsPair:
    question: str
    options: JobOptions

In [11]:
# options에 key값 -> 오탈자 유발가능
question_options_pairs: list[QuestionOptionsPair] = [
    QuestionOptionsPair(
        question="본인이 생각하기에 자신의 강점은 무엇인가요?", 
        options=JobOptions(frontend="창의력", backend="논리적 사고", cloud="효율적 관리", security="분석 능력"),
    ),
    QuestionOptionsPair(
        question="작업 스타일에 대한 선호도는 어떤가요?",
        options=JobOptions(frontend="빠르게 결과물을 확인할 수 있는 작업", backend="깊이 있는 문제 해결을 요구하는 작업", cloud="시스템 전체를 설계하고 관리하는 작업", security="보안 위협을 예측하고 대응하는 작업"),
    ),
    QuestionOptionsPair(
        question="어떤 유형의 문제를 해결할 때 가장 만족감을 느끼나요?",
        options=JobOptions(frontend="세세한 배열", backend="시스템 오류를 해결하거나 최적화하는 문제", cloud="인프라를 구축하거나 확장 가능한 시스템 설계", security="취약점 발견"),
    ),
    QuestionOptionsPair(
        question="학습 속도의 관심사",
        options=JobOptions(frontend="최신 트렌드나 기술을 배우는 것을 좋아하나요?", backend="깊이 있는 이론과 구조를 배우는 것을 선호하나요?", cloud="최신 트렌드나 기술을 배우는 것을 좋아하나요?", security="깊이 있는 이론과 구조를 배우는 것을 선호하나요?")
    )
]
# 이제 정적으로 정의된거임!

In [14]:
JobOptions.__annotations__.keys()

dict_keys(['frontend', 'backend', 'cloud', 'security'])

In [16]:
user_job_choice = {}
for key in JobOptions.__annotations__.keys():
    user_job_choice[key] =0

In [ ]:
import time

user_job_choice = {}
for key in JobOptions.__annotations__.keys():
    user_job_choice[key] =0

job_key_list = list(user_job_choice.keys())

for pair in question_options_pairs:
    print(pair)
    print(f"Q: {pair.question}")
    for index, (key, value) in enumerate(pair.options.__dict__.items()):
        print(f"    {index+1}. {value}")
    print('system: 위 4지선다에서 질문의 번호 하나를 입력해주세요.')
    time.sleep(1)
    user_answer = int(input())
    user_job_choice[job_key_list[user_answer -1]] += 1
    print('system: 내가 선택한 번호는? ',user_answer)

max_job = max(user_job_choice, key=user_job_choice.get)
print(f'내가 너에게 추천하는 직종은! {max_job}이야!')
        

QuestionOptionsPair(question='본인이 생각하기에 자신의 강점은 무엇인가요?', options=JobOptions(frontend='창의력', backend='논리적 사고', cloud='효율적 관리', security='분석 능력'))
Q: 본인이 생각하기에 자신의 강점은 무엇인가요?
    1. 창의력
    2. 논리적 사고
    3. 효율적 관리
    4. 분석 능력
system: 위 4지선다에서 질문의 번호 하나를 입력해주세요.
system: 내가 선택한 번호는?  1
QuestionOptionsPair(question='작업 스타일에 대한 선호도는 어떤가요?', options=JobOptions(frontend='빠르게 결과물을 확인할 수 있는 작업', backend='깊이 있는 문제 해결을 요구하는 작업', cloud='시스템 전체를 설계하고 관리하는 작업', security='보안 위협을 예측하고 대응하는 작업'))
Q: 작업 스타일에 대한 선호도는 어떤가요?
    1. 빠르게 결과물을 확인할 수 있는 작업
    2. 깊이 있는 문제 해결을 요구하는 작업
    3. 시스템 전체를 설계하고 관리하는 작업
    4. 보안 위협을 예측하고 대응하는 작업
system: 위 4지선다에서 질문의 번호 하나를 입력해주세요.
system: 내가 선택한 번호는?  2
QuestionOptionsPair(question='어떤 유형의 문제를 해결할 때 가장 만족감을 느끼나요?', options=JobOptions(frontend='세세한 배열', backend='시스템 오류를 해결하거나 최적화하는 문제', cloud='인프라를 구축하거나 확장 가능한 시스템 설계', security='취약점 발견'))
Q: 어떤 유형의 문제를 해결할 때 가장 만족감을 느끼나요?
    1. 세세한 배열
    2. 시스템 오류를 해결하거나 최적화하는 문제
    3. 인프라를 구축하거나 확장 가능한 시스템 설계
    4. 취약점 발견
system:

In [20]:
print(f"해당 {max_job}직무에 대해 어느정도 알고 계시는지 몇 가지 질문을 드려보겠습니다.")

해당 backend직무에 대해 어느정도 알고 계시는지 몇 가지 질문을 드려보겠습니다.


In [29]:
difficulty_job_template = PromptTemplate.from_template(
    f"""
    사용자가 {{max_job}}직무에 대해 얼마나 알고있는지 파악할거야.
    {{max_job}}직무를 수행하기 위한 기초/ 중급/ 고급 수준에서 각각 하나씩 질문을 뽑아줘.
    각 질문에 대한 답을 4지선다로 뽑아줘.
    반환 형태를 list[str] 형태를 JSON 형식로 출력하세요.

    입력:
    프론트엔드

    생성된 질문:
    ["질문1: \예시: \정답:", "질문2: \예시: \정답:", "질문3: \예시: \정답:"]

    입력:
    {{max_job}}

    키워드:
    """.strip()
)

In [30]:
def get_generated_input(max_job: str) -> str:
    result = difficulty_job_template | gpt4o | JsonOutputParser()
    generated_job_questions = result.invoke({"max_job": max_job})
    return generated_job_questions

In [31]:
user_input = max_job
joblevel_qna = get_generated_input(user_input)
joblevel_qna

['질문1: 백엔드 개발에서 RESTful API란 무엇인가요? \n예시: A) 사용자 인터페이스 설계 기법 B) 서버와 클라이언트 간의 통신을 위한 아키텍처 스타일 C) 데이터베이스 관리 시스템 D) 서버 보안 프로토콜 \n정답: B',
 '질문2: 데이터베이스에서 인덱스를 사용하는 주된 이유는 무엇인가요? \n예시: A) 데이터의 중복을 방지하기 위해 B) 데이터의 무결성을 보장하기 위해 C) 데이터 검색 속도를 향상시키기 위해 D) 데이터 저장 공간을 절약하기 위해 \n정답: C',
 '질문3: 마이크로서비스 아키텍처의 장점 중 하나는 무엇인가요? \n예시: A) 모놀리식 구조로 인한 유지보수 용이성 B) 서비스 간의 강한 결합 C) 개별 서비스의 독립적 배포 및 확장성 D) 단일 코드베이스의 단순성 \n정답: C']

In [38]:
qnas = []
for qna in joblevel_qna:
    qnas.append(qna.split('\n'))

In [36]:
qnas

[['질문1: 백엔드 개발에서 RESTful API란 무엇인가요? ',
  '예시: A) 사용자 인터페이스 설계 기법 B) 서버와 클라이언트 간의 통신을 위한 아키텍처 스타일 C) 데이터베이스 관리 시스템 D) 서버 보안 프로토콜 ',
  '정답: B'],
 ['질문2: 데이터베이스에서 인덱스를 사용하는 주된 이유는 무엇인가요? ',
  '예시: A) 데이터의 중복을 방지하기 위해 B) 데이터의 무결성을 보장하기 위해 C) 데이터 검색 속도를 향상시키기 위해 D) 데이터 저장 공간을 절약하기 위해 ',
  '정답: C'],
 ['질문3: 마이크로서비스 아키텍처의 장점 중 하나는 무엇인가요? ',
  '예시: A) 모놀리식 구조로 인한 유지보수 용이성 B) 서비스 간의 강한 결합 C) 개별 서비스의 독립적 배포 및 확장성 D) 단일 코드베이스의 단순성 ',
  '정답: C']]